# Designing your own sentiment analysis tool

While there are a lot of tools that will automatically give us a sentiment of a piece of text, we learned that they don't always agree! Let's design our own to see both how these tools work internally, along with how we can test them to see how well they might perform.

**I've cleaned the dataset up a bit.**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


<p class="reading-options">
  <a class="btn" href="/investigating-sentiment-analysis/designing-your-own-sentiment-analysis-tool">
    <i class="fa fa-sm fa-book"></i>
    Read online
  </a>
  <a class="btn" href="/investigating-sentiment-analysis/notebooks/Designing your own sentiment analysis tool.ipynb">
    <i class="fa fa-sm fa-download"></i>
    Download notebook
  </a>
  <a class="btn" href="https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/investigating-sentiment-analysis/notebooks/Designing your own sentiment analysis tool.ipynb" target="_new">
    <i class="fa fa-sm fa-laptop"></i>
    Interactive version
  </a>
</p>

## Training on tweets

Let's say we were going to analyze the sentiment of tweets. If we had a list of tweets that were scored positive vs. negative, we could see which words are usually associated with positive scores and which are usually associated with negative scores.

Luckily, we have **Sentiment140** - http://help.sentiment140.com/for-students - a list of 1.6 million tweets along with a score as to whether they're negative or positive. We'll use it to build our own machine learning algorithm to see separate positivity from negativity.

### Read in our data

In [2]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/data_set/sts-test_data_set/training.csv", names=['polarity','id','date','Query','About','text'],encoding='latin-1')
df.drop(['id','date','Query','About'],inplace=True,axis=1)
df.head()

,polarity,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


It isn't a very complicated dataset. `polarity` is whether it's positive or not, `text` is the text of the tweet itself.

How many rows do we have?

In [3]:
df=df.sample(n=1000000,replace=True)
df=df[0:500000]

How many **positive** tweets compared to how many **negative** tweets?

In [4]:
df.polarity.value_counts()

4    250170
0    249830
Name: polarity, dtype: int64

## Train our algorithm


### Vectorize our tweets

Create a `TfidfVectorizer` and use it to vectorize our tweets. Since we don't have all the time in the world, we should probably use `max_features` to only take a selection of terms - how about 1000 for now?

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
vectorizer = TfidfVectorizer(max_features=1000)
vectors = vectorizer.fit_transform(df.text)
words_df = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names())
words_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10,100,11,12,15,1st,20,2day,2nd,30,...,yesterday,yet,yo,you,your,yours,yourself,youtube,yummy,yup
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Setting up our variables

Because we want to fit in with all the other progammers, we need to create two variables: one called `X` and one called `y`.

`X` is all of our **features**, the things we use to predict positive or negative. That's going to be our words.

`y` is all of our **labels**, the positive or negative rating. We'll use the `polarity` column for that.

In [7]:
X = words_df
y = df.polarity

### Picking an algorithm

What kind of algorithm do we want? Who knows, we don't know anything about machine learning! **Let's just pick ALL OF THEM.**

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

### Training our algorithms

When we teach our algorithm about what a positive or a negative tweet looks like, this is called **training**. Training can take different amounts of time based on what kind of algorithm you are using.

In [9]:
%%time
# Create and train a logistic regression
logreg = LogisticRegression(C=1e9, solver='lbfgs', max_iter=1000)
#logreg.fit(X, y)

CPU times: user 23 µs, sys: 5 µs, total: 28 µs
Wall time: 32.9 µs


In [10]:
%%time
# Create and train a random forest classifier
forest = RandomForestClassifier(n_estimators=50)
#forest.fit(X, y)

CPU times: user 34 µs, sys: 7 µs, total: 41 µs
Wall time: 45.1 µs


In [11]:
%%time
# Create and train a linear support vector classifier (LinearSVC)
svc = LinearSVC()
#svc.fit(X, y)

CPU times: user 0 ns, sys: 17 µs, total: 17 µs
Wall time: 20 µs


In [12]:
%%time
# Create and train a multinomial naive bayes classifier (MultinomialNB)
bayes = MultinomialNB()
#bayes.fit(X, y)

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 10.5 µs


**How long did each take to train?** How much faster were some compared to others?

## Use our models

Now that we've trained our models, **they can try to predict whether some content is positive or negative**.

### Preparing the data

**Add a few more sentences below.** They should be a mix of positive and negative. They can be boring, they can be exciting, they can be short, they can be long.

In [14]:
# Create some test data

pd.set_option("display.max_colwidth", 200)

unknown = pd.DataFrame({'content': [
    "I love love love love this kitten",
    "I hate hate hate hate this keyboard",
    "I'm not sure how I feel about toast",
    "Did you see the baseball game yesterday?",
    "The package was delivered late and the contents were broken",
    "Trashy television shows are some of my favorites",
    "I'm seeing a Kubrick film tomorrow, I hear not so great things about it.",
    "I find chirping birds irritating, but I know I'm not the only one",
]})
unknown

,content
0,I love love love love this kitten
1,I hate hate hate hate this keyboard
2,I'm not sure how I feel about toast
3,Did you see the baseball game yesterday?
4,The package was delivered late and the contents were broken
5,Trashy television shows are some of my favorites
6,"I'm seeing a Kubrick film tomorrow, I hear not so great things about it."
7,"I find chirping birds irritating, but I know I'm not the only one"


First we need to **vectorizer** our sentences into numbers, so the algorithm can understand them.

Our algorithm only knows **certain words.** Run `vectorizer.get_feature_names()` to show you the list of the words it knows.

In [15]:
print(vectorizer.get_feature_names())

['10', '100', '12', '15', '1st', '20', '30', '3d', '5th', 'able', 'about', 'absolutely', 'access', 'account', 'actually', 'add', 'adorable', 'advice', 'after', 'afternoon', 'again', 'ago', 'agree', 'agreed', 'ah', 'ahead', 'ahh', 'ahhh', 'airport', 'album', 'all', 'almost', 'alone', 'already', 'also', 'always', 'am', 'amazing', 'american', 'amp', 'an', 'and', 'another', 'answer', 'any', 'anymore', 'anyone', 'anything', 'anyway', 'app', 'apparently', 'apple', 'are', 'aren', 'argh', 'around', 'as', 'asap', 'asleep', 'ass', 'assignment', 'at', 'ate', 'australia', 'available', 'aw', 'awake', 'awards', 'away', 'awesome', 'aww', 'awww', 'baby', 'back', 'background', 'bad', 'badly', 'band', 'be', 'beach', 'beautiful', 'because', 'bed', 'been', 'beer', 'before', 'behind', 'being', 'believe', 'best', 'bet', 'better', 'big', 'bike', 'birthday', 'bit', 'black', 'blog', 'body', 'boo', 'book', 'books', 'bored', 'boring', 'both', 'bought', 'bout', 'boy', 'break', 'breakfast', 'bright', 'bring', 'bro

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Usually when we use the vectorizer, we write code like this:
    
```python
vectors = vectorizer.fit_transform(....)
```

Which both learns all the words **and** counts them. In this case **we already have the list of words we know, we only want to count them.** So instead of `.fit_transform`, we just use `.transform`:

```python
unknown_vectors = vectorizer.transform(unknown.content)
unknown_words_df = ......
```

Finish making your `unknown_words_df` in the cell below.

In [16]:
# Put it through the vectoriser

# transform, not fit_transform, because we already learned all our words
unknown_vectors = vectorizer.transform(unknown.content)
unknown_words_df = pd.DataFrame(unknown_vectors.toarray(), columns=vectorizer.get_feature_names())
unknown_words_df.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,10,100,12,15,1st,20,30,3d,5th,able,...,yep,yes,yesterday,yet,you,your,yours,youtube,yummy,yup
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.511531,0.0,0.241284,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


Confirm `unknown_words_df` is 11 rows and 2,000 columns.

In [ ]:
unknown_words_df.shape

(8, 1000)

### Predicting with our models

To make a prediction for each of our sentences, you can use `.predict` with each of our models. For example, it would look like this for linear regression:

```python
unknown['pred_logreg'] = logreg.predict(unknown_words_df)
```

To add the prediction for logistic regression, you'd run similar `.predict` code, which will give you a `0` (negative) or a `1` (positive). A difference between the two is that for logistic regression, you can **also ask for the probability that the sentence is in the `1` category** instead of just simply the category. To do that, you use this code:

```python
unknown['pred_logreg_prob'] = linreg.predict_proba(unknown_words_df)[:,1]
```

**Add new columns for each of the models you trained.** If the model has a `.predict_proba`, add that as a column as well. 

* **Tip:** Tab is helpful for knowing whether `.predict_proba` is an option.
* **Tip:** Don't forget the `[:,1]` after `.predict_proba`, it means "give me the probability for category `1`

In [17]:
# Predict using all our models. 

# Logistic Regression predictions + probabilities
unknown['pred_logreg'] = logreg.predict(unknown_words_df)
unknown['pred_logreg_proba'] = logreg.predict_proba(unknown_words_df)[:,1]

# Random forest predictions + probabilities
unknown['pred_forest'] = forest.predict(unknown_words_df)
unknown['pred_forest_proba'] = forest.predict_proba(unknown_words_df)[:,1]

# SVC predictions
unknown['pred_svc'] = svc.predict(unknown_words_df)

# Bayes predictions + probabilities
unknown['pred_bayes'] = bayes.predict(unknown_words_df)
unknown['pred_bayes_proba'] = bayes.predict_proba(unknown_words_df)[:,1]

In [ ]:
unknown

,content,pred_logreg,pred_logreg_proba,pred_forest,pred_forest_proba,pred_svc,pred_bayes,pred_bayes_proba
0,I love love love love this kitten,1,0.950442,1,0.848665,1,1,0.747222
1,I hate hate hate hate this keyboard,0,0.009593,0,0.000000,0,0,0.122383
2,I'm not sure how I feel about toast,0,0.180952,0,0.240000,0,0,0.416819
3,Did you see the baseball game yesterday?,1,0.615063,1,0.660000,1,1,0.509662
4,The package was delivered late and the contents were broken,0,0.058171,0,0.460000,0,0,0.219788
5,Trashy television shows are some of my favorites,0,0.330293,0,0.440000,0,1,0.534234
6,"I'm seeing a Kubrick film tomorrow, I hear not so great things about it.",1,0.558548,0,0.260000,1,1,0.533493
7,"I find chirping birds irritating, but I know I'm not the only one",0,0.060122,0,0.440000,0,0,0.295739


### Questions

* What do the numbers mean? What's the difference between a 0 and a 1? A 0.5? Negative numbers?
* Were there any sentences where the classifiers seemed to disagree about? How do you feel about the amount they disagree? 
* What's the difference between using a 0/1 to talk about sentiment compared to 0-1? When might you use one compared to another?
* What's the difference between the linear regression model and the other models we're using? Why might it fit or not fit?
* Between 0-1, what range do you think counts as "negative," "positive" and "neutral"?
* Does the variation in scores reflect the variation you would see among people? Or is it better or worse?

## Testing our models

We can actually see **which model performs the best!** Remember how we trained our models on tweets? We can ask each model about each tweet, and see if it gets the right answer.

In [13]:
df.head()

,polarity,text
632633,0,"had a good night, but sad tomorrow is andys la..."
667070,0,I got a cold...or something..my throat is so s...
799781,0,"This month, O'Neal said he asked Fawcett to ma..."
1230833,4,Enjoying the first day of summer with loads of...
113566,0,Official advisory on whether I'm an arsehole a...


Our original dataframe is a list of many, many tweets. We turned this into `X` - vectorized words - and `y` - whether the tweet is negative or positive.

Before we used `.fit(X, y)` to train on all of our data. Instead, **we can test our models** by doing a test/train split and see if the predictions match the actual labels.

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

In [16]:
%%time

#print("Training logistic regression")
#logreg.fit(X_train, y_train)

#print("Training random forest")
#forest.fit(X_train, y_train)

print("Training SVC")
svc.fit(X_train, y_train)

print("Training Naive Bayes")
bayes.fit(X_train, y_train)

Training SVC
Training Naive Bayes
CPU times: user 13.4 s, sys: 143 ms, total: 13.5 s
Wall time: 14.3 s


### Confusion matrices

To see how well they did, we'll use a ["confusion matrix"](https://en.wikipedia.org/wiki/Confusion_matrix) for each one. I think confusion matrices are called that because they are confusing.

In [17]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

#### Logistic Regression

In [18]:
y_true = y_test
y_pred = logreg.predict(X_test)
print(classification_report(y_test,y_pred))
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
print(pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names))
print(pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0))

              precision    recall  f1-score   support

           0       0.77      0.75      0.76     62744
           4       0.76      0.78      0.77     62256

    accuracy                           0.76    125000
   macro avg       0.76      0.76      0.76    125000
weighted avg       0.76      0.76      0.76    125000

             Predicted negative  Predicted positive
Is negative               47004               15740
Is positive               13709               48547
             Predicted negative  Predicted positive
Is negative            0.749139            0.250861
Is positive            0.220204            0.779796


#### Random forest

In [36]:
y_true = y_test
y_pred = forest.predict(X_test)
print(classification_report(y_test,y_pred))
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
print(pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names))
print(pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0))

              precision    recall  f1-score   support

           0       0.74      0.77      0.76     12479
           4       0.76      0.74      0.75     12521

    accuracy                           0.75     25000
   macro avg       0.75      0.75      0.75     25000
weighted avg       0.75      0.75      0.75     25000

             Predicted negative  Predicted positive
Is negative                9626                2853
Is positive                3300                9221
             Predicted negative  Predicted positive
Is negative            0.771376            0.228624
Is positive            0.263557            0.736443


#### SVC

In [19]:
y_true = y_test
y_pred = svc.predict(X_test)
print(classification_report(y_test,y_pred))
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
print(pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names))
print(pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0))

              precision    recall  f1-score   support

           0       0.77      0.75      0.76     62744
           4       0.75      0.78      0.77     62256

    accuracy                           0.76    125000
   macro avg       0.76      0.76      0.76    125000
weighted avg       0.76      0.76      0.76    125000

             Predicted negative  Predicted positive
Is negative               46783               15961
Is positive               13597               48659
             Predicted negative  Predicted positive
Is negative            0.745617            0.254383
Is positive            0.218405            0.781595


#### Multinomial Naive Bayes

In [38]:
y_true = y_test
y_pred = bayes.predict(X_test)
print(classification_report(y_test,y_pred))
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['negative', 'positive'])
print(pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names))
print(pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0))

              precision    recall  f1-score   support

           0       0.75      0.75      0.75     12479
           4       0.75      0.74      0.75     12521

    accuracy                           0.75     25000
   macro avg       0.75      0.75      0.75     25000
weighted avg       0.75      0.75      0.75     25000

             Predicted negative  Predicted positive
Is negative                9347                3132
Is positive                3194                9327
             Predicted negative  Predicted positive
Is negative            0.749018            0.250982
Is positive            0.255091            0.744909


## Review

If you find yourself unsatisfied with a tool, you can try to build your own! This is exactly what we tried to do, using the **Sentiment140 dataset** and several machine learning algorithms.

Sentiment140 is a database of tweets that come pre-labeled with positive or negative sentiment, assigned automatically by presence of a `:)` or `:(`.  Our first step was using a **vectorizer** to convert the tweets into numbers a computer could understand.

After that, we build four different **models** using different machine learning algorithms. Each one was fed a list of each tweet's **features** - the words - and each tweet's **label** - the sentiment - in the hopes that later it could predict labels if given a new tweets. This process of teaching the algorithm is called **training**.

In order to test our algorithms, we split our data into sections - **train** and **test** datasts. You teach the algorithm with the first group, and then ask it for predictions on the second set. You can then compare its predictions to the right answers using a **confusion matrix**.

Although **different algorithms took different amounts of time to train**, they all ended up with about 70-75% accuracy.

## Discussion topics

* Which models performed the best? Were there big differences?
* Do you think it's more important to be sensitive to negativity or positivity? Do we want more positive things incorrectly marked as negative, or more negative things marked as positive?
* They all had very different training times. Which ones offer the best combination of performance and not making you wait around for an hour?
* If you have a decent algorithm that trains more quickly, that could that mean about feature selection or the size of your training set? Why did we use `max_features=` and `df.sample`?
* Is 75% accuracy good?
* Do your feelings change if the performance is described as "incorrect one out of every four times?"
* What would your accuracy be for a random guess?
* How do you feel about sentiment analysis?
* How do you feel about [this piece from the UpShot](https://www.nytimes.com/interactive/2017/02/28/upshot/trump-sounds-different-tone-in-first-address-to-congress.html) that uses [the Emotional Lexicon](http://saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm)?
* What would you feel comfortable using our sentiment classifier for?